In [1]:
# only for taurus
import os
print(os.getcwd())
os.chdir("notebooks")
print(os.getcwd())

/data/horse/ws/jori152b-medinf/KP_MedInf
/data/horse/ws/jori152b-medinf/KP_MedInf/notebooks


In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# File paths and separator
DATA_PATH_stages = "../data/extracted/kdigo_stages_measured.csv"
DATA_PATH_labs = "../data/extracted/labs_original.csv"
DATA_PATH_labs_extended = "../data/extracted/labs_extended.csv"
DATA_PATH_labs_new = "../data/extracted/labs_new.csv"
# DATA_PATH_vitals = "../data/extracted/vitals-kdigo_stages_measured.csv"
DATA_PATH_vitals = "../data/extracted/vitals.csv"
# DATA_PATH_vents = "../data/extracted/vents-vasopressor-sedatives-kdigo_stages_measured.csv"
DATA_PATH_vents = "../data/extracted/vents_vasopressor_sedatives.csv"
# DATA_PATH_detail = "../data/extracted/icustay_detail-kdigo_stages_measured.csv"
DATA_PATH_detail = "../data/extracted/icustay_detail.csv"
DATA_PATH_heightweight = "../data/extracted/heightweight.csv"
DATA_PATH_calcium = "../data/extracted/calcium.csv"
DATA_PATH_inr_max = "../data/extracted/inr_max.csv"
SEPARATOR = ";"

# Constants
IMPUTE_EACH_ID = False
IMPUTE_COLUMN = False
TESTING = False
TEST_SIZE = 0.05
SPLIT_SIZE = 0.2
MAX_DAYS = 35
CLASS1 = True
ALL_STAGES = False
MAX_FEATURE_SET = True
FIRST_TURN_POS = True
TIME_SAMPLING = True
SAMPLING_INTERVAL = '6H'
RESAMPLE_LIMIT = 16
MOST_COMMON = False
IMPUTE_METHOD = 'most_frequent'
FILL_VALUE = 0
ADULTS_MIN_AGE = 18
ADULTS_MAX_AGE = 120
NORMALIZATION = 'min-max'
HOURS_AHEAD = 48
NORM_TYPE = 'min_max'
RANDOM = 42

def filter_by_length_of_stay(X):
    drop_list = []
    long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)

    for icustay_id, is_long in long_stays.items():
        if is_long:
            max_time = X[X['icustay_id'] == icustay_id]['charttime'].max() - pd.to_timedelta(MAX_DAYS, unit='D')
            X = X[~((X['icustay_id'] == icustay_id) & (X['charttime'] < max_time))]

    short_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) < (HOURS_AHEAD/24))
    drop_list = short_stays[short_stays].index.tolist()

    X = X[~X.icustay_id.isin(drop_list)]
    return X

In [21]:
# Load datasets
print("Loading datasets...")
X = pd.read_csv(DATA_PATH_stages, sep=SEPARATOR)
X.drop(["aki_stage_creat", "aki_stage_uo"], axis=1, inplace=True)
X = X.dropna(how='all', subset=['creat', 'uo_rt_6hr', 'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage'])
X['charttime'] = pd.to_datetime(X['charttime'])

print(len(X))
print(X['aki_stage'].value_counts())

dataset_detail = pd.read_csv(DATA_PATH_detail, sep=SEPARATOR)
# original data
# dataset_detail.drop(['dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity', 
#                      'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 
#                      'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay'], axis=1, inplace=True)
# out data
dataset_detail.drop(['dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity', 
                     'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 
                     'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'ethnicity_grouped'], axis=1, inplace=True)

# dataset_labs = pd.read_csv(DATA_PATH_labs, sep=SEPARATOR)
# dataset_labs = dataset_labs.dropna(subset=['charttime']).dropna(subset=dataset_labs.columns[4:], how='all')
# dataset_labs['charttime'] = pd.to_datetime(dataset_labs['charttime'])
# dataset_labs = dataset_labs.sort_values(by=['icustay_id', 'charttime'])
# dataset_labs.drop(['albumin_min', 'albumin_max','bilirubin_min', 'bilirubin_max','bands_min', 'bands_max',
#                    'lactate_min', 'lactate_max','platelet_min', 'platelet_max','ptt_min', 'ptt_max', 
#                    'inr_min', 'inr_max', 'pt_min', 'pt_max'], axis = 1, inplace = True)
# # Calculate mean for each pair and drop original columns
# column_pairs = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
#                 ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
#                 ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
#                 ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
#                 ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
#                 ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
#                 ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
#                 ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
#                 ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max')]

# for min_col, max_col in column_pairs:
#     try:
#         mean_col = min_col.rsplit('_', 1)[0] + '_mean'
#         dataset_labs[mean_col] = dataset_labs[[min_col, max_col]].mean(axis=1)
#         dataset_labs.drop([min_col, max_col], axis=1, inplace=True)
#     except:
#         pass

dataset_labs_extended = pd.read_csv(DATA_PATH_labs_extended, sep=SEPARATOR)
dataset_labs_extended = dataset_labs_extended.dropna(subset=['charttime']).dropna(subset=dataset_labs_extended.columns[4:], how='all')
dataset_labs_extended['charttime'] = pd.to_datetime(dataset_labs_extended['charttime'])
dataset_labs_extended = dataset_labs_extended.sort_values(by=['icustay_id', 'charttime'])

column_pairs_extended = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
                ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
                ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
                ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
                ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
                ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
                ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
                ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
                ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max'), 
                ('gfr_min', 'gfr_max'), ('phosphate_min', 'phosphate_max'),('uric_acid_min', 'uric_acid_max'), 
                ('calcium_min', 'calcium_max')]


for min_col, max_col in column_pairs_extended:
    mean_col = min_col.rsplit('_', 1)[0] + '_mean'
    dataset_labs_extended[mean_col] = dataset_labs_extended[[min_col, max_col]].mean(axis=1)
    dataset_labs_extended.drop([min_col, max_col], axis=1, inplace=True)
dataset_labs_extended.drop(['gfr_mean'], axis=1, inplace=True)

dataset_vitals = pd.read_csv(DATA_PATH_vitals, sep=SEPARATOR)
dataset_vents = pd.read_csv(DATA_PATH_vents, sep=SEPARATOR)
dataset_vitals.drop(["heartrate_min", "heartrate_max", "sysbp_min", "sysbp_max", "diasbp_min", "diasbp_max",
                        'meanbp_min', 'meanbp_max', 'tempc_min', 'tempc_max', "resprate_min", "resprate_max", 
                        "spo2_min", "spo2_max", "glucose_min", "glucose_max"], axis=1, inplace=True)
dataset_vitals['charttime'] = pd.to_datetime(dataset_vitals['charttime'])
dataset_vents['charttime'] = pd.to_datetime(dataset_vents['charttime'])
dataset_vitals = dataset_vitals.dropna(subset=dataset_vitals.columns[4:], how='all')
dataset_vitals = dataset_vitals.sort_values(by=['icustay_id', 'charttime'])
dataset_vents = dataset_vents.sort_values(by=['icustay_id', 'charttime'])

dataset_heightweight = pd.read_csv(DATA_PATH_heightweight, sep=SEPARATOR)
dataset_heightweight = dataset_heightweight.dropna(subset=['icustay_id', 'height_first', 'weight_first'], how='all')
dataset_heightweight = dataset_heightweight.sort_values(by=['icustay_id'])

dataset_calcium = pd.read_csv(DATA_PATH_calcium, sep=SEPARATOR)
dataset_calcium.drop(["hadm_id"], axis=1, inplace=True)
dataset_calcium['charttime'] = pd.to_datetime(dataset_calcium['charttime'])
dataset_calcium = dataset_calcium.sort_values(by=['icustay_id', 'charttime'])
dataset_calcium.drop(["subject_id"], axis=1, inplace=True)

dataset_inr_max = pd.read_csv(DATA_PATH_inr_max, sep=SEPARATOR)
dataset_inr_max.drop(["hadm_id", "subject_id"], axis=1, inplace=True)
dataset_inr_max = dataset_inr_max.sort_values(by=['icustay_id'])



# Merge datasets
if MAX_FEATURE_SET:
    # X = X.merge(dataset_labs, on=["icustay_id", "charttime"], how="outer")
    X = X.merge(dataset_labs_extended, on=["icustay_id", "charttime"], how="outer")
    X = X.merge(dataset_vitals, on=["icustay_id", "charttime", "subject_id", "hadm_id"], how="outer")
    X = X.merge(dataset_vents, on=["icustay_id", "charttime"], how="outer")
    X.drop(["subject_id"], axis=1, inplace=True)
    X = X.merge(dataset_calcium, on=["icustay_id", "charttime"], how="outer")

Loading datasets...
3737147
aki_stage
0    3000795
2     353266
3     207759
1     175327
Name: count, dtype: int64


In [22]:
X.columns

Index(['icustay_id', 'charttime', 'creat', 'uo_rt_6hr', 'uo_rt_12hr',
       'uo_rt_24hr', 'aki_stage', 'hadm_id', 'aniongap_mean', 'albumin_mean',
       'bands_mean', 'bicarbonate_mean', 'bilirubin_mean', 'creatinine_mean',
       'chloride_mean', 'glucose_mean_x', 'hematocrit_mean', 'hemoglobin_mean',
       'lactate_mean', 'platelet_mean', 'potassium_mean', 'ptt_mean',
       'inr_mean', 'pt_mean', 'sodium_mean', 'bun_mean', 'wbc_mean',
       'phosphate_mean', 'uric_acid_mean', 'calcium_mean', 'heartrate_mean',
       'sysbp_mean', 'diasbp_mean', 'meanbp_mean', 'resprate_mean',
       'tempc_mean', 'spo2_mean', 'glucose_mean_y', 'vent', 'vasopressor',
       'sedative', 'calcium'],
      dtype='object')

In [20]:
dataset_detail.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admission_age'], dtype='object')

In [15]:
dataset_calcium.columns

Index(['subject_id', 'icustay_id', 'charttime', 'calcium'], dtype='object')

In [23]:
print("Filtering patients by age and length of stay...")
# Filtering patients by age and length of stay
dataset_detail = dataset_detail[dataset_detail['admission_age'] >= ADULTS_MIN_AGE]
adults_icustay_id_list = dataset_detail['icustay_id'].unique()
X = X[X.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id', 'charttime'])

X = filter_by_length_of_stay(X)
dataset_detail = dataset_detail[dataset_detail.icustay_id.isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])

Filtering patients by age and length of stay...


In [10]:
label = ['aki_stage']
skip = ['icustay_id', 'charttime', 'aki_stage']
discrete_feat = ['sedative', 'vasopressor', 'vent', 'hadm_id']
skip.extend(discrete_feat)    
numeric_feat = list(X.columns.difference(skip))

In [8]:
# save to csv
X.to_csv('../data/analysis/data_preprocessed_filtered_before_resampling.csv', index=False)

In [20]:
X = pd.read_csv('../data/analysis/data_preprocessed_extended__filtered_before_resampling.csv')

## looped dataset creation

In [14]:
SAMPLING_INTERVALS = ['1H', '2H', '3H', '4H', '6H']


In [11]:
dataset_detail = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
try:
    # theirs
    dataset_detail = pd.get_dummies(dataset_detail, columns=['gender', 'ethnicity_grouped'])
except:
    #ours
    dataset_detail = pd.get_dummies(dataset_detail, columns=['gender'])
dataset_detail.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)

In [12]:
dataset_detail.columns

Index(['icustay_id', 'admission_age', 'gender_F', 'gender_M'], dtype='object')

In [24]:
# saving every sampling interval
# Resampling
for SAMPLING_INTERVAL in SAMPLING_INTERVALS:
    if TIME_SAMPLING:
        
        # Set index and group by 'icustay_id' before resampling
        X_resampled = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
        
        # Resample and aggregate features
        if MAX_FEATURE_SET:
            X_discrete = X_resampled[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
        X_numeric = X_resampled[numeric_feat].mean()
        X_label = X_resampled['aki_stage'].max()

        print("Merging sampled features")
        try:
            X_resampled = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
        except:
            X_resampled = pd.concat([X_numeric, X_label], axis=1).reset_index()



    # Forward fill again after resampling
    X_resampled['aki_stage'] = X_resampled.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)



    # Ensure binary values (convert any positive number to 1)
    X_resampled['aki_stage'] = (X_resampled['aki_stage'] > 0).astype(int)

    # Shifting labels
    shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
    X_resampled['aki_stage'] = X_resampled.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
    X_resampled = X_resampled.dropna(subset=['aki_stage'])

    # Merging not time-dependent data

    X_resampled = X_resampled.merge(dataset_detail, on='icustay_id')
    X_resampled = X_resampled.merge(dataset_heightweight, on='icustay_id')
    X_resampled = X_resampled.merge(dataset_inr_max, on='icustay_id')

    # If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
    X_resampled = X_resampled.fillna(FILL_VALUE) 


    # Save preprocessed data
    X_resampled.to_csv(f'../data/preprocessed/preprocessed_data_extended_{SAMPLING_INTERVAL}.csv', index=False)




/tmp/ipykernel_834289/3017786234.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X_resampled = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


## normal ds creation

In [9]:
# Resampling
if TIME_SAMPLING:
    
    # Set index and group by 'icustay_id' before resampling
    X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
    
    # Resample and aggregate features
    if MAX_FEATURE_SET:
        X_discrete = X[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
    X_numeric = X[numeric_feat].mean()
    X_label = X['aki_stage'].max()

    print("Merging sampled features")
    try:
        X = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
    except:
        X = pd.concat([X_numeric, X_label], axis=1).reset_index()



# Forward fill again after resampling
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)



# Ensure binary values (convert any positive number to 1)
X['aki_stage'] = (X['aki_stage'] > 0).astype(int)

# Shifting labels
shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
X = X.dropna(subset=['aki_stage'])

# Merging not time-dependent data
dataset_detail = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
try:
    # theirs
    dataset_detail = pd.get_dummies(dataset_detail, columns=['gender', 'ethnicity_grouped'])
except:
    #ours
    dataset_detail = pd.get_dummies(dataset_detail, columns=['gender'])
dataset_detail.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)
X = X.merge(dataset_detail, on='icustay_id')
X = X.merge(dataset_heightweight, on='icustay_id')
X = X.merge(dataset_inr_max, on='icustay_id')

# If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
X = X.fillna(FILL_VALUE) 


/tmp/ipykernel_3091928/2742523973.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


Merging sampled features


In [10]:

# Save preprocessed data
X.to_csv('../data/preprocessed/preprocessed_data.csv', index=False)